In [ ]:
Model Building
to Build a model, I'll try different algorithms and evaluate their performances by RMSE, Finally I'll apply a stacking method which combines all the algorithms to see if I get a better result
from sklearn.model_selection import train_test_split
​
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
​
from sklearn.ensemble import StackingRegressor
​
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error
from numpy import mean
from numpy import std
​
from matplotlib import pyplot
.iloc
X=train.iloc[:,:-1]
y=train.iloc[:,-1:]
# first I define a stacking function
# first I define a stacking function
def get_stacking():
    # define the base models
    level0 = list()
    level0.append(('lr', LinearRegression()))
    level0.append(('knn', KNeighborsRegressor()))
    level0.append(('cart', DecisionTreeRegressor()))
    level0.append(('svm', SVR()))
    level0.append(('ada', AdaBoostRegressor()))
    level0.append(('xgb', XGBRegressor()))
    level0.append(('rf', RandomForestRegressor()))
    level0.append(('la', Lasso()))
    level0.append(('rd', Ridge()))
    # define meta learner model
    level1 = LinearRegression()
    # define the stacking ensemble
    model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
    return model
# next I define a function to evaluate different models
# next I define a function to evaluate different models
def evaluate_model(model):
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    neg_scores = cross_val_score(model, X, y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
    scores= np.sqrt(abs(neg_scores))
    return scores
function to create a dictionary of desired algorithms
# here I define function to create a dictionary of desired algorithms
def get_models():
    models = dict()
    models['lr'] = LinearRegression()
    models['knn'] = KNeighborsRegressor()
    models['cart'] = DecisionTreeRegressor()
    models['svm'] = SVR()
    models['ada'] = AdaBoostRegressor()
    models['xgb']= XGBRegressor()
    models['rf']= RandomForestRegressor()
    models['la']= Lasso()
    models['rd']= Ridge()
    models['stacking'] = get_stacking()
    return models
# creating a dictionary of models
models = get_models()
​
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()
It's obvious that stacking improved the preformance of the model, also linear regression and ridge algorithms perform quite good.
the performance by stacking model might even progress if we tune parameters for each algorithm beforehand.
Now it's time to predict sales for our test data and upload our submission to the contest.
_
model=get_stacking()
model.fit(X, y)
y_pred=model.predict(test)
submission=pd.read_csv('sample_submission')
Importnant Note:
after submitting my prediction to the solution checker for the first time, I got a notice telling submission file includes some negative values, which is quite non-sense since I am predicting sales figures.
I googled the problem and found out that the application of Linear regression and Ridge might cause to get negative values in my prediction. why? since the algorithms are based on y=ax+b, which extrapolate predictions, that means it is possible that the interception for the equation to be negative, that means some values somewhere, some how would be predicted negative.
How to solve this issue? well, there are two approaches, one way is to use another algorithm, and the other is to apply log transformation to independent variable (target value). I tried the second approch and then inversed both transformation (normalization, log transformation) for predicted values.
y_pred_inv
y_pred_inv=norm_scaler_3.inverse_transform(y_pred.reshape(-1,1))
y_pred_inv_exp=np.exp(y_pred_inv)
submission.head()
Item_Identifier	Outlet_Identifier	Item_Outlet_Sales
0	FDW58	OUT049	7.125195
1	FDW14	OUT017	7.312422
2	NCN55	OUT010	6.419935
3	FDQ58	OUT017	7.789930
4	FDY38	OUT027	8.545310
_inv_exp
submission=submission.drop(['Item_Outlet_Sales'], axis=1)
submission['Item_Outlet_Sales']=y_pred_inv_exp
submission.to_csv('submission.csv', index=False)
after submitting the submission file I've got the score of 1232.9415921557, which is not bad for a model without any hyperparameter tuning, however, it is possible to improve the performance of this model by tuning it's parameters.